In [0]:
spark.sql("USE CATALOG hive_metastore")

In [0]:
%sql
show databases

In [0]:
from pyspark.sql.functions import sum,round
orders = spark.sql("SELECT order_id,order_date FROM retail_test.orders")
orderitemsdf = spark.sql("SELECT order_id, total FROM retail_test.orderItems")
ordersdf = orders.join(orderitemsdf, orders.order_id == orderitemsdf.order_id,"inner").drop("order_id").groupBy("order_date").agg(round(sum("total"),2)).alias("total").sort("order_date", ascending=False)
display(ordersdf)

# Get Revenue by category

In [0]:
%sql
select * from retail_test.orderItems limit 5

In [0]:
%sql
select * from retail_test.products limit 5

In [0]:
orderitemsdf = spark.sql("SELECT product_id, total FROM retail_test.orderItems")
productsdf = spark.sql("select id, category_id from retail_test.products")
prodorderitemsdf = productsdf.join(orderitemsdf, productsdf.id == orderitemsdf.product_id,"inner").drop("product_id")
categorydf = spark.sql("select id, name from retail_test.categories")
finaldf = prodorderitemsdf.join(categorydf, prodorderitemsdf.category_id == categorydf.id,"inner").drop(prodorderitemsdf.category_id).groupBy(categorydf.name).agg(round(sum("total"),2).alias("total"))
finaldf.display()




In [0]:
ordersdf = spark.sql("select * from retail_test.orders limit 5")
ordersdf.display()


In [0]:
ordersdf = spark.sql("select order_Id, customer_id from retail_test.orders")
customersdf = spark.sql("select id, fname,lname from retail_test.customers")
windowshopers = customersdf.join(ordersdf, customersdf.id == ordersdf.customer_id,"left").filter(ordersdf.customer_id.isNull()).select(customersdf.id,customersdf.fname,customersdf.lname)
display(windowshopers)

# Window functions

In [0]:
%sql
use catalog hive_metastore

In [0]:
%sql
use database retail_test


In [0]:


orders = spark.sql("select order_id, order_date from retail_test.orders")
orderitemsdf = spark.sql("select order_id, total from retail_test.orderItems")
from pyspark.sql.functions import sum,round
ordersoidf = orders.join(orderitemsdf, orders.order_id == orderitemsdf.order_id,"inner").drop(orders.order_id).groupBy("order_id").agg(round(sum("total"),2).alias("TotalRevenue"))
display(ordersoidf)



In [0]:
ordersfinal = orders.join(ordersoidf, orders.order_id == ordersoidf.order_id,"inner").drop(ordersoidf.order_id).select(orders.order_id,orders.order_date,ordersoidf.TotalRevenue)
ordersfinal.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

window_spec = Window.partitionBy("order_date").orderBy(desc("TotalRevenue"))
rankdata = ordersfinal.withColumn("rank",rank().over(window_spec))
display(rankdata)

In [0]:
rankdataFinaldf = rankdata.filter(rankdata.rank == 1).drop("rank")
display(rankdataFinaldf)

In [0]:
from pyspark.sql.functions import sum,round
ordersniitemsdata = orders.join(orderitemsdf, orders.order_id == orderitemsdf.order_id,"inner").groupBy("order_date").agg(round(sum(orderitemsdf.total),2).alias("Revenue"))
ordersniitemsdata.display()


In [0]:
from pyspark.sql.functions import lag, col,lead
from pyspark.sql.window import Window

finnalcomparisondat = ordersniitemsdata.withColumn("previous_sales", lag(col("Revenue")).over(Window.orderBy(desc("order_date")))).withColumn("Next_Sales", lead(col("Revenue")).over(Window.orderBy(desc("order_date"))))
lagnleaddata = finnalcomparisondat.select("order_date","previous_sales","Revenue","Next_Sales").display()
display(lagnleaddata)


# Union All/Union

In [0]:
orders

In [0]:
%sql
select * from global_temp.completed_orders_gbl